<a href="https://colab.research.google.com/github/MarcelaFerreiraR/Assistente_Academico_Gemini/blob/main/Assistente_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Célula 1: Instalações
!pip install streamlit langchain langchain-google-genai langchain-community pypdf faiss-cpu pyngrok nest_asyncio -q
print("Bibliotecas instaladas (incluindo langchain-community)!")

In [ ]:
# Célula 2: Imports e Configuração da API Key
import streamlit as st
import os
import google.generativeai as genai
from google.colab import userdata
from pyngrok import ngrok
import nest_asyncio

In [ ]:
# Aplica patch para asyncio se necessário
nest_asyncio.apply()

In [ ]:
# Pegar a API key do Colab Secrets
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    if not api_key:
        raise ValueError("API Key não encontrada ou vazia.")
    os.environ['GOOGLE_API_KEY'] = api_key
    genai.configure(api_key=api_key)
    print("✅ Chave da API do Google configurada com sucesso!")
except Exception as e:
    print(f"🚨 Erro ao configurar a API Key: {e}")
    print("🚨 Verifique se você adicionou a 'GOOGLE_API_KEY' aos Secrets do Colab (ícone de chave 🔑 na barra lateral) e ativou 'Notebook access'.")

In [ ]:
## Baixar Bibliotecas
%%writefile app.py
import streamlit as st
import os
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import time
import sys

print("--- Log 1: Script iniciado ---", flush=True)

# --- Configuração da API Key ---
api_key_in_app = os.getenv("GOOGLE_API_KEY")
genai_configured = False
if api_key_in_app:
    try:
        genai.configure(api_key=api_key_in_app)
        genai_configured = True
        print("--- Log 2: genai.configure SUCESSO ---", flush=True)
    except Exception as e:
        st.error(f"Erro configurando API: {e}")
        print(f"--- Log 3: ERRO genai.configure: {e} ---", flush=True)
else:
    st.error("API Key não encontrada no ambiente.")
    print("--- Log 4: API Key NÃO encontrada ---", flush=True)


# --- Funções Auxiliares ---
print("--- Log 5: Definindo funções auxiliares (com código)... ---", flush=True)

def process_pdf(uploaded_file):
    print("--- DEBUG process_pdf: Iniciando ---", flush=True)
    if uploaded_file is not None:
        temp_file_path = f"./{uploaded_file.name}"
        with open(temp_file_path, "wb") as f:
            f.write(uploaded_file.getbuffer())
        print(f"--- DEBUG process_pdf: PDF salvo temporariamente em {temp_file_path} ---", flush=True)

        loader = PyPDFLoader(temp_file_path)
        try:
            print("--- DEBUG process_pdf: Carregando e dividindo páginas... ---", flush=True)
            pages = loader.load_and_split()
            print(f"--- DEBUG process_pdf: {len(pages)} páginas carregadas ---", flush=True)
        except Exception as e:
            print(f"--- DEBUG process_pdf: ERRO no PyPDFLoader: {e} ---", flush=True)
            st.error(f"Erro ao carregar o PDF com PyPDFLoader: {e}")
            if os.path.exists(temp_file_path): os.remove(temp_file_path)
            return None

        print("--- DEBUG process_pdf: Dividindo em chunks... ---", flush=True)
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000, chunk_overlap=200, length_function=len, add_start_index=True
        )
        texts = text_splitter.split_documents(pages)
        print(f"--- DEBUG process_pdf: Texto dividido em {len(texts)} chunks ---", flush=True)

        if os.path.exists(temp_file_path): os.remove(temp_file_path)
        print("--- DEBUG process_pdf: Arquivo temporário removido. Retornando chunks. ---", flush=True)
        return texts
    print("--- DEBUG process_pdf: Nenhum arquivo enviado. Retornando None. ---", flush=True)
    return None

def create_vector_store(text_chunks):
    print("--- DEBUG create_vector_store: Iniciando ---", flush=True)
    if not text_chunks:
        print("--- DEBUG create_vector_store: Nenhum chunk recebido. Retornando None. ---", flush=True)
        st.warning("Nenhum chunk de texto para vetorizar.")
        return None
    try:
        print("--- DEBUG create_vector_store: Criando embeddings... ---", flush=True)
        embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
        print("--- DEBUG create_vector_store: Construindo FAISS... ---", flush=True)
        vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
        print("--- DEBUG create_vector_store: FAISS construído com SUCESSO. Retornando store. ---", flush=True)
        return vector_store
    except Exception as e:
        print(f"--- DEBUG create_vector_store: ERRO: {e} ---", flush=True)
        st.error(f"Erro ao criar o banco de vetores: {e}")
        return None

def get_conversational_chain():
    print("--- DEBUG get_conversational_chain: Iniciando ---", flush=True)
    prompt_template_str = """
    Você é um assistente de IA especializado em responder perguntas com base em documentos acadêmicos (contexto) fornecidos.
    Responda à pergunta da forma mais detalhada e precisa possível usando APENAS o contexto fornecido abaixo.
    Cite trechos relevantes do contexto para embasar sua resposta sempre que possível.
    Se a resposta não puder ser encontrada no contexto, diga explicitamente: "Com base no contexto fornecido, não consigo responder a essa pergunta."
    Não invente informações.

    Contexto:
    {context}

    Pergunta:
    {question}

    Resposta detalhada:
    """
    # O importante é que ele use {context} e {question}

    print("--- DEBUG get_conversational_chain: Inicializando ChatGoogleGenerativeAI... ---", flush=True)
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.2, convert_system_message_to_human=True)

    print("--- DEBUG get_conversational_chain: Criando PromptTemplate... ---", flush=True)
    prompt = PromptTemplate(template=prompt_template_str, input_variables=["context", "question"])

    print("--- DEBUG get_conversational_chain: Carregando load_qa_chain... ---", flush=True)
    chain = load_qa_chain(
        llm=model,
        chain_type="stuff",
        prompt=prompt,
        document_variable_name="context",
        verbose=False
    )
    print("--- DEBUG get_conversational_chain: Cadeia criada com SUCESSO. Retornando chain. ---", flush=True)
    return chain

print("--- Log 6: Funções auxiliares definidas (com código) ---", flush=True)

# --- Interface Streamlit ---
print("--- Log 7: Renderizando UI completa... ---", flush=True)
st.set_page_config(page_title="🎓 Assistente Mestrado (Teste UI)", layout="wide")
st.header("🎓 Assistente de Mestrado com Gemini")
st.markdown("Faça upload de um PDF...")

# --- Estado da Sessão ---
print("--- Log 8: Inicializando session_state... ---", flush=True)
if "vector_store" not in st.session_state: st.session_state.vector_store = None
if "pdf_processed_name" not in st.session_state: st.session_state.pdf_processed_name = None
if "messages" not in st.session_state: st.session_state.messages = []
print("--- Log 9: session_state inicializado ---", flush=True)


# --- Barra Lateral ---
print("--- Log 10: Renderizando sidebar... ---", flush=True)
with st.sidebar:
    st.subheader("Seus Documentos")
    pdf_file = st.file_uploader("Carregue seu PDF aqui", type="pdf", accept_multiple_files=False)

    # --- LÓGICA DO BOTÃO PROCESSAR PDF - ATIVADA ---
    if st.button("Processar PDF") and pdf_file is not None:
        print(f"--- DEBUG Botão Processar PDF CLICADO para: {pdf_file.name} ---", flush=True)
        if st.session_state.pdf_processed_name != pdf_file.name:
             st.session_state.vector_store = None
             st.session_state.messages = []
             with st.spinner(f"Processando '{pdf_file.name}'... Isso pode levar alguns minutos."):
                text_chunks = process_pdf(pdf_file)
                if text_chunks:
                    st.session_state.vector_store = create_vector_store(text_chunks)
                    if st.session_state.vector_store:
                        st.session_state.pdf_processed_name = pdf_file.name
                        st.success(f"PDF '{pdf_file.name}' processado!")
                        print(f"--- DEBUG PDF '{pdf_file.name}' processado com sucesso ---", flush=True)
                    else:
                        st.error("Falha ao criar banco de vetores (FAISS). Verifique os logs.")
                        st.session_state.pdf_processed_name = None
                        print(f"--- DEBUG Falha em create_vector_store ---", flush=True)
                else:
                    st.error("Não foi possível extrair texto ou processar o PDF.")
                    st.session_state.pdf_processed_name = None
                    print(f"--- DEBUG Falha em process_pdf ---", flush=True)
        else:
             st.info(f"PDF '{pdf_file.name}' já foi processado anteriormente.")


    # --- ELIF  ---
    elif pdf_file is None and st.session_state.vector_store is not None:

         if st.button("Limpar PDF Carregado"):
              print("--- DEBUG Botão Limpar PDF CLICADO ---", flush=True)
              st.session_state.vector_store = None
              st.session_state.pdf_processed_name = None
              st.session_state.messages = []
              st.rerun()
    # --- FIM DO ELIF ---

    # --- Exibição do Status (Alinhado com if/elif) ---

    if st.session_state.pdf_processed_name:
        st.success(f"PDF Ativo: {st.session_state.pdf_processed_name}")
    else:
        st.info("Nenhum PDF processado.")

print("--- Log 11: Sidebar renderizada ---", flush=True)


# --- Área Principal de Chat ---
print("--- Log 12: Renderizando área principal... ---", flush=True)
st.subheader("Faça sua Pergunta")
user_question = st.text_input("Sua pergunta sobre o PDF:", key="user_input", disabled=not st.session_state.vector_store)

# --- LÓGICA DO BOTÃO ENVIAR PERGUNTA - ATIVADA ---
if st.button("Enviar Pergunta", disabled=not st.session_state.vector_store or not user_question):
    print(f"--- DEBUG Botão Enviar Pergunta CLICADO com pergunta: {user_question} ---", flush=True)
    # --- DESCOMENTAR AS LINHAS ABAIXO ---
    if st.session_state.vector_store and user_question:
        with st.spinner("Pensando... 🧠"):
            vector_store = st.session_state.vector_store
            try:
                # 1. Buscar documentos similares (Retrieval)
                print(f"--- DEBUG QA: Buscando documentos similares para: {user_question} ---", flush=True)
                docs = vector_store.similarity_search(user_question, k=5)
                print(f"--- DEBUG QA: Encontrados {len(docs)} documentos ---", flush=True)

                # 2. Chamar a cadeia de QA (Generation)
                print("--- DEBUG QA: Obtendo cadeia de QA... ---", flush=True)
                chain = get_conversational_chain()
                print("--- DEBUG QA: Executando cadeia de QA... ---", flush=True)
                response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
                answer = response["output_text"]
                print("--- DEBUG QA: Resposta recebida do LLM ---", flush=True)

                # 3. Mostrar a resposta
                st.subheader("Resposta do Assistente:")
                st.markdown(answer)

            except Exception as e:
                error_message = f"Ocorreu um erro ao processar sua pergunta: {e}"
                print(f"--- DEBUG QA: ERRO: {error_message} ---", flush=True)
                st.error(error_message)
                if "API key" in str(e).lower() or "permission" in str(e).lower() or "quota" in str(e).lower():
                     st.error("Erro de API ou Cota: Verifique sua chave, permissões e limites de uso.")
                else:
                     st.error("Um erro inesperado ocorreu.")

    elif not st.session_state.vector_store:
        st.warning("Por favor, processe um PDF antes de fazer perguntas.")
    elif not user_question:
         st.warning("Por favor, digite sua pergunta.")
# --- FIM DA LÓGICA DO BOTÃO ENVIAR PERGUNTA ---

print("--- Log 13: Área principal renderizada ---", flush=True)

# --- Rodapé ---
# ... (código do rodapé) ...
print("--- Log 14: Fim do script ---", flush=True)
sys.stdout.flush()

In [ ]:
# Célula 4: Authtoken do Ngrok

# Imports necessários para esta célula
from google.colab import userdata
from pyngrok import ngrok
import os

# Mata processos anteriores se existirem (útil ao re-executar a célula)
!killall ngrok
!killall streamlit

# Define a porta padrão do Streamlit
port = 8501

# --- CONFIGURAÇÃO DO NGTOK AUTHTOKEN ---
try:
    authtoken = userdata.get('NGROK_AUTHTOKEN')
    if not authtoken:
        raise ValueError("NGROK_AUTHTOKEN não encontrado ou vazio nos Secrets.")
    # Configura o authtoken no pyngrok
    ngrok.set_auth_token(authtoken)
    print("✅ Authtoken do Ngrok configurado com sucesso!")
except Exception as e:
    print(f"🚨 Erro ao obter/configurar o Authtoken do Ngrok: {e}")
    print("🚨 Verifique se você adicionou 'NGROK_AUTHTOKEN' aos Secrets do Colab (🔑) e ativou 'Notebook access'.")
    print("🚨 Crie uma conta e obtenha seu token em https://dashboard.ngrok.com/get-started/your-authtoken")


# --- Tenta iniciar o Ngrok e o Streamlit ---
try:
    # Inicia o ngrok em background para expor a porta escolhida
    public_url = ngrok.connect(port)
    print("="*50)
    print(f"✅ Aplicativo Streamlit rodando! Acesse em: {public_url}")
    print("="*50)

    # Executa o Streamlit em background usando nohup
    # O '&' no final garante que ele rode em background
    !nohup streamlit run app.py --server.port {port} --server.enableCORS=false --server.enableXsrfProtection=false &

except Exception as e:
    print(f"🚨 Erro ao iniciar o ngrok ou Streamlit: {e}")
    # Tenta desconectar se ngrok conectou mas streamlit falhou
    active_tunnels = ngrok.get_tunnels()
    for tunnel in active_tunnels:
       ngrok.disconnect(tunnel.public_url)
       print(f"Tunnel {tunnel.public_url} desconectado.")

# A célula continuará rodando para manter o servidor Streamlit e o túnel ngrok ativos.
# Para parar o aplicativo, você precisa interromper a execução desta célula (botão de parar).